In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from gtts import gTTS
import os
from sklearn.metrics import confusion_matrix

In [6]:
gif_path = '/Users/prithvik/Documents/USC/Assignments/EE541/EE541 Project/India.gif' 

In [13]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from gtts import gTTS
from torchvision import models
import os

# Load the pre-trained model and set device
ResNet_dict = torch.load('tuned_resnet_model.pth',map_location=torch.device('cpu'))
ResNet = models.resnet50()
num_features = ResNet.fc.in_features
ResNet.fc = nn.Linear(num_features, 29)
ResNet.load_state_dict(ResNet_dict)

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load and preprocess the GIF file
gif_path = '/Users/prithvik/Documents/USC/Assignments/EE541/EE541 Project/India.gif'  # Replace with the path to your GIF file
gif = Image.open(gif_path)
frame_rate = gif.info['duration'] / 1000 
frames = []
for frame in range(0, gif.n_frames, 69):
    gif.seek(frame)
    image = gif.convert('RGB')
    image = transform(image).unsqueeze(0)
    frames.append(image)

# Make predictions for each frame and transcribe into speech
predicted_letters = []
for image in frames:
    with torch.no_grad():
        output = ResNet(image)
        _, predicted = torch.max(output, 1)
    predicted_letter = chr(ord('A') + predicted.item())  # Convert label to letter
    predicted_letters.append(predicted_letter)

# Convert predicted letters into a sentence
sentence = ' '.join(predicted_letters)

# Generate speech from the sentence using gTTS
tts = gTTS(text=sentence, lang='en')
audio_path = 'output.mp3'  # Path to save the generated audio file
tts.save(audio_path)

# Play the generated audio
os.system(f'afplay {audio_path}')  # Works on macOS, change command for other platforms if needed


0

In [12]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load the pre-trained model
ResNet_dict = torch.load('tuned_resnet_model.pth',map_location=torch.device('cpu'))
ResNet = models.resnet50()
num_features = ResNet.fc.in_features
ResNet.fc = nn.Linear(num_features, 29)
ResNet.load_state_dict(ResNet_dict)

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load and preprocess the image
image_path = '/Users/prithvik/Documents/USC/Assignments/EE541/Project/data/asl_alphabet_train/asl_alphabet_train/Y/Y6.jpg'  # Replace with the path to your image
image = Image.open(image_path)
image = transform(image).unsqueeze(0)

# Make a prediction
with torch.no_grad():
    output = ResNet(image)
    _, predicted = torch.max(output, 1)

# Convert the predicted label to the corresponding letter
label_mapping = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
predicted_letter = label_mapping[predicted.item()]

# Print the predicted letter
print('Predicted letter:', predicted_letter)


Predicted letter: I


In [1]:
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models
from gtts import gTTS
from playsound import playsound

# Load the pre-trained model
ResNet_dict = torch.load('tuned_resnet_model.pth',map_location=torch.device('cpu'))
ResNet = models.resnet50()
num_features = ResNet.fc.in_features
ResNet.fc = nn.Linear(num_features, 29)
ResNet.load_state_dict(ResNet_dict)

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load label mapping
label_mapping = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

# Start video capture from webcam
cap = cv2.VideoCapture(0)

# Initialize variables for letter prediction
current_letter = None
prev_letter = None

while True:
    # Read frame from webcam
    ret, frame = cap.read()

    # Convert the frame to PIL image
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Preprocess the image
    image = transform(image).unsqueeze(0)

    # Make a prediction
    with torch.no_grad():
        output = ResNet(image)
        _, predicted = torch.max(output, 1)

    # Get the predicted letter
    current_letter = label_mapping[predicted.item()]

    # Play letter audio if it's a new letter
    if current_letter != prev_letter:
        prev_letter = current_letter

        # Convert letter to speech
        tts = gTTS(text=current_letter, lang='en')
        tts.save('temp.mp3')

        # Play the audio
        playsound('temp.mp3')

    # Display the frame
    cv2.imshow('ASL Transcription', frame)

    # Check for 'q' key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()


NameError: name 'nn' is not defined